### Pytorch 모델 파일을 tensorflow 파일로 변환, 이미지 입력 후 prediction하고, 후처리하여 출력

아래 코드를 실행시켜 학습이 완료된 pytorch 모델파일(best.pt)을 tensorflow 모델파일(saved model)로 변환합니다.

>{가상환경 경로명}\Scripts\python.exe {yolov5-master 경로명}\export.py --weights {yolov5-master 경로명}\runs\train\exp\weights\best.pt --include saved_model

{yolov5-master 경로명}\runs\train\exp\weights 폴더로 들어가면 tensorflow 모델로 변환된 'saved_model' 폴더가 생성된 것을 확인할 수 있습니다.

'saved_model' 폴더를 실습파일이 있는 경로(현재 보고 있는 이 주피터 노트북 파일이 있는 경로)로 복사해 옵니다.

In [164]:
import cv2
import numpy as np
import tensorflow as tf

In [165]:
#tf saved_model로 변환된 모델 파일 로드
model = tf.saved_model.load("best_saved_model")

In [166]:
#모델의 입출력 구조 확인 → 입력은 (1, 640, 640, 3), 출력은 (1, 25200, 7)임을 알 수 있음
for k, v in model.signatures.items():
    print(k, v)

serving_default ConcreteFunction signature_wrapper(*, x)
  Args:
    x: float32 Tensor, shape=(1, 640, 640, 3)
  Returns:
    {'output_0': <1>}
      <1>: float32 Tensor, shape=(1, 25200, 7)


이미지 하나를 로드해 모델에 입력으로 넣고 prediction한 결과를 봅니다.

In [167]:
#테스트할 이미지를 행렬로 로드
src = cv2.imread("test_images/test_image1.jpg")
# src_converted = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
src_converted = src.copy()
src_converted.shape # -> (2268, 4032, 3) 4032 x 2268 사이즈의 3채널 컬러이미지

(2268, 4032, 3)

In [168]:
#우리가 만든 모델은 (1, 640, 640, 3)을 입력으로 받으므로, 그에 맞게 resize & reshape을 진행
# ※입력에 차원이 하나 더 붙어있는 이유는 배치 때문으로, resize 후 reshape을 통해 변환

src_resized = cv2.resize(src_converted, (640, 640))
src_resized.shape # -> (640, 640, 3)

(640, 640, 3)

In [169]:
src_resized = src_resized.reshape((1, 640, 640, 3))
src_resized.shape

(1, 640, 640, 3)

In [170]:
#추론 실행
pred = model(src_resized)
pred[0].shape # -> TensorShape([1, 25200, 7])  *numpy가 아니라 tf 프레임웍 포맷에 맞춰 출력되므로 주의

TensorShape([1, 25200, 7])

In [171]:
#제일 앞에 있는 1차원은 의미 없고 나머지 차원이 중요 -> 25200행 * 7열
#25200행 : 입력에 따라 추론한 25200개의 객체검출 결과
#7열 : 행별 결과내용 → [cx, cy, w, h, obj conf, c1 conf, c2 conf]
# 0~3번째 열은 검출한 바운딩 박스 정보
# 4번째 열은 바운딩 박스 안에 객체가 있을 확률
# 5번째 열은 검출한 객체가 1번 클래스(도로표지판)일 확률
# 6번째 열은 검출한 객체가 2번 클래스(신호등)일 확률

pred = pred[0][0].numpy() #필요없는 차원들은 제거하고 (25200, 7)만 선택해 numpy 행렬로 변환한 후 pred에 덮어 씌움
pred

array([[1.28766662e-02, 1.36250481e-02, 3.42174172e-02, ...,
        9.71086621e-02, 6.47929549e-01, 9.23326194e-01],
       [1.20488359e-02, 4.27273149e-03, 2.74170656e-02, ...,
        7.37711787e-04, 2.15256751e-01, 9.96479690e-01],
       [1.05029512e-02, 6.13262737e-03, 5.18705063e-02, ...,
        1.68289155e-01, 1.19463485e-02, 6.53255880e-01],
       ...,
       [9.91511464e-01, 9.86274660e-01, 4.74171452e-02, ...,
        5.47400923e-05, 1.00116342e-01, 7.89171696e-01],
       [9.78893578e-01, 9.74242687e-01, 1.24012023e-01, ...,
        2.12199520e-04, 2.59680033e-01, 7.13711619e-01],
       [9.84046757e-01, 9.77106988e-01, 8.94882232e-02, ...,
        9.94062648e-05, 8.60639036e-01, 6.90861046e-02]], dtype=float32)

In [172]:
#pred의 4번째 행(pred[:, 4])를 기준으로 정렬하면 객체가 있을 확률이 높은 순서대로 결과를 볼 수 있음
rank = np.argsort(-pred[:, 4])
pred = pred[rank]
pred

array([[4.0967584e-01, 5.8125472e-01, 2.7013250e-08, ..., 1.0000000e+00,
        9.3931783e-28, 1.0000000e+00],
       [7.7487034e-01, 4.9375001e-01, 1.6793365e-07, ..., 1.0000000e+00,
        4.9680103e-23, 1.0000000e+00],
       [1.3681720e-01, 6.1875004e-01, 1.3790456e-02, ..., 1.0000000e+00,
        3.4181184e-14, 1.0000000e+00],
       ...,
       [7.9233074e-01, 8.0809051e-01, 4.2428114e-02, ..., 0.0000000e+00,
        8.2703694e-10, 9.9998450e-01],
       [1.4374630e-01, 8.4375137e-01, 2.7069103e-02, ..., 0.0000000e+00,
        3.3525866e-05, 9.9996841e-01],
       [1.8732123e-02, 8.8125020e-01, 1.8719088e-02, ..., 0.0000000e+00,
        2.4660142e-07, 5.6903887e-01]], dtype=float32)

추론한 바운딩 박스 정보를 원본 이미지에 그린 후 결과를 봅니다.

In [173]:
def filterCandidates(pred, conf_thres=0.8):
    candidate_list = None
    for item in pred:
        if item[4] >= conf_thres:
            this_sum = 0
            for i in range(0, 2):
                this_sum += item[i]*item[i]
            this_l2 = np.sqrt(this_sum)
            item = np.append(item, [this_l2])
            if candidate_list is None:
                candidate_list = np.array([item])
            else:
                candidate_list = np.vstack([candidate_list, item])
    rank = np.argsort(-candidate_list[:, -1])
    candidate_list = candidate_list[rank]
    return candidate_list

def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

def nms(candidate_list, iou_thres = 0.45, merge_thres = 0.75, print_iou=False):
    result = candidate_list.copy()
    idx_list_to_delete = []
    for idx, item in enumerate(candidate_list):
        if idx+1 < len(candidate_list):
            next_item = candidate_list[idx+1]
            box1 = (item[0] - (item[2]/2), item[1] - (item[3]/2), item[2], item[3])
            box2 = (next_item[0] - (next_item[2]/2), next_item[1] - (next_item[3]/2), next_item[2], next_item[3])
            iou = IoU(box1, box2)
            if print_iou:
                print(iou)
            if iou > merge_thres: #bbox를 합쳐버린다~!
                x = min(box1[0], box2[0])
                y = min(box1[1], box2[1])
                w = max(box1[2], box2[2])
                h = max(box1[3], box2[3])
                replace_val = [x + (w/2), y + (h/2), w, h, item[4], item[5], item[6], item[7]]
                result[idx] = replace_val
                idx_list_to_delete.append(idx+1)
            elif iou > iou_thres:
                idx_list_to_delete.append(idx+1)
    result = np.delete(result, idx_list_to_delete, axis=0)
    return result

conf_thres = 0.8
candidate_list = filterCandidates(pred, 0.8)
for _ in range(10):
    candidate_list = nms(candidate_list)
candidate_list.shape

(634, 8)

In [174]:
class_names = ["sign", "light"]
dst = cv2.resize(src, (1600, int((1600*src.shape[0])/src.shape[1]))) #이미지가 너무 크므로 리사이즈한다.
for bbox in candidate_list:
    if bbox[5] < conf_thres and bbox[6] < conf_thres:
        continue

    if bbox[5] > bbox[6]:
        class_name = class_names[0]
        class_conf = bbox[5]
        color = (0, 255, 0)
    else:
        class_name = class_names[1]
        class_conf = bbox[6]
        color = (0, 0, 255)
    
    cx = int(bbox[0] * dst.shape[1])
    cy = int(bbox[1] * dst.shape[0])
    w = int(bbox[2] * dst.shape[1])
    h = int(bbox[3] * dst.shape[0])
    x = int(cx - (w/2))
    y = int(cy - (h/2))

    if w < 10 or h < 10:
        continue

    dst = cv2.rectangle(dst, (x, y, w, h), color, 1)
    dst = cv2.putText(dst, "{}".format(int(bbox[4]*100)), (x, y), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()


1.0
1.0
1.0
0.8853479623794556
0.9999997615814209
1.0
1.0
1.0
1.0
1.0
1.0
0.9999656677246094
0.9867151379585266
0.998822033405304
1.0
0.9752887487411499
0.9999850988388062
1.0
1.0
1.0
1.0
1.0
0.9542067646980286
0.9672896862030029
0.9820789694786072
1.0
0.9743748307228088
1.0
1.0
1.0
0.9752551913261414
1.0
1.0
0.9999998807907104
1.0
1.0
0.9975595474243164
0.9993603825569153
0.9999998807907104
1.0
0.9998830556869507
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8156186938285828
0.9970274567604065
1.0
1.0
0.9934491515159607
0.9801448583602905
1.0
1.0
0.9999997615814209
0.9999994039535522
0.9999895095825195
1.0
0.9844010472297668
0.9999046325683594
0.9999995231628418
0.9976873397827148
0.8118270635604858
0.999484658241272
0.999954104423523
0.9998229146003723
0.9981535077095032
1.0
0.8923943638801575
1.0
1.0
1.0
0.9999998807907104
1.0
0.9999841451644897
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999843835830688
1.0
0.9999814033508301
1.0
0.9999951124191284
1.0
0.998710632324

Lessons

1. 물체감지는 라벨링 작업에 시간이 많이 걸리므로, 이번 실습은 일반적으로 공개된 데이터를 활용해 진행할 수 밖에 없었다.

2. 현업에서 실제 활용할 목적으로 모델을 만들고자 한다면, 수집한 이미지 데이터에 맞는 라벨링 작업이 필수로 요구된다.